### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Cystic_Fibrosis/GSE38267'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Sample Characteristics Dictionary as per STEP 1 output
sample_characteristics = {0: ['disease state: cystic fibrosis (CF)', 
                              'disease state: pulmonary arterial hypertension (PAH)', 
                              'disease state: healthy'], 
                          1: ['tissue: whole blood'], 
                          2: ['gender: male', 'gender: female']} 

# Determine if gene expression data is available
is_gene_available = True  # Since the study describes microarray analysis which typically measures gene expression

# Identify rows for variables in sample characteristics dictionary
trait_row = 0 if len({v.split(': ')[1] for v in sample_characteristics[0]}) > 1 else None
age_row = None  # Age information appears to be not available in the sample characteristics dictionary
gender_row = 2 if len({v.split(': ')[1] for v in sample_characteristics[2]}) > 1 else None

# Define conversion functions
def convert_trait(value):
    try:
        value = value.split(': ')[1].lower()
        if value == 'cystic fibrosis (cf)':
            return 1
        elif value in ['pulmonary arterial hypertension (pah)', 'healthy']:
            return 0
        else:
            return None
    except:
        return None

def convert_age(value):
    try:
        value = value.split(': ')[1]
        return float(value) if value.replace('.', '', 1).isdigit() else None
    except:
        return None

def convert_gender(value):
    try:
        value = value.split(': ')[1].lower()
        if value == 'male':
            return 1
        elif value == 'female':
            return 0
        else:
            return None
    except:
        return None

# Save cohort information
save_cohort_info('GSE38267', './preprocessed/Cystic_Fibrosis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction if trait information is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Cystic_Fibrosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Cystic_Fibrosis/trait_data/GSE38267.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify the keys for probe ID and gene symbols
identifier_key = 'ProbeName'
gene_symbol_key = 'GeneName'

# Print column names for debugging
print("Gene Annotation Columns:", gene_annotation.columns)

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = gene_annotation[[identifier_key, gene_symbol_key]]
gene_mapping = get_gene_mapping(gene_mapping, identifier_key, gene_symbol_key)

# 3. Apply the mapping to get the gene expression dataframe
gene_data.index = gene_data.index.astype(str)  # Ensure index is string type
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Cystic_Fibrosis/gene_data/GSE38267.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Cystic_Fibrosis')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE38267', './preprocessed/Cystic_Fibrosis/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Cystic_Fibrosis/GSE38267.csv'
    unbiased_merged_data.to_csv(csv_path)
